## Image Segmentation

In [12]:
%matplotlib inline
import keras
from keras.models import *
from keras.layers import *
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.datasets import mnist
from keras.utils import np_utils
from keras.preprocessing.image import img_to_array
from skimage.transform import resize

In [13]:
%run Dataset.py
%run Models/FCN32.py
%run Models/FCN8.py
%run Models/VGGUnet.py
%run Models/VGGSegnet.py
%run metrics.py

In [3]:
SEG_PATH = './CityScapes_Dataset/segmentations'
IMG_PATH = './CityScapes_Dataset/rawimgs'
IMAGE_ORDERING = 'channels_first'
N_CLASSES = 30
INPUT_HEIGHT = 224
INPUT_WIDTH = 224

In [4]:
train_gen = DataGenerator(img_path=IMG_PATH, seg_path=SEG_PATH, split='small_train',
                  n_classes=30,
                  input_width=224, input_height=224,
                  output_width=112, output_height=112,
                  imgNorm='sub_mean',
                  ordering='channels_first',
                  batch_size=16
                  )

In [5]:
val_gen = DataGenerator(img_path=IMG_PATH, seg_path=SEG_PATH, split='small_val',
                  n_classes=30,
                  input_width=224, input_height=224,
                  output_width=112, output_height=112,
                  imgNorm='sub_mean',
                  ordering='channels_first',
                  batch_size=16
                  )

In [6]:
colormap, labelmap = create_label_dicts()

In [7]:
labelmap

{0: 'unlabeled',
 1: 'dynamic',
 2: 'ground',
 3: 'road',
 4: 'sidewalk',
 5: 'parking',
 6: 'rail track',
 7: 'building',
 8: 'wall',
 9: 'fence',
 10: 'guard rail',
 11: 'bridge',
 12: 'tunnel',
 13: 'pole',
 14: 'polegroup',
 15: 'traffic light',
 16: 'traffic sign',
 17: 'vegetation',
 18: 'terrain',
 19: 'sky',
 20: 'person',
 21: 'rider',
 22: 'car',
 23: 'truck',
 24: 'bus',
 25: 'caravan',
 26: 'trailer',
 27: 'train',
 28: 'motorcycle',
 29: 'bicycle'}

### Image Segmentation - Check Preprocessed Datasets

In [8]:
print(
    len(os.listdir('CityScapes_Dataset/rawimgs/small_train')),
    len(os.listdir('CityScapes_Dataset/segmentations/small_train')),
    len(os.listdir('CityScapes_Dataset/rawimgs/small_val')),
    len(os.listdir('CityScapes_Dataset/segmentations/small_val'))
)

1500 1500 500 500


### Check Models are Working

### Model Configurations

In [16]:
model = VGGUnet(N_CLASSES, INPUT_HEIGHT, INPUT_WIDTH)

metrics = ['accuracy', iou, iou_flat, dice, dice_flat]


model.compile(optimizer='adam', 
              loss=dice_loss,  
              metrics=metrics)

In [17]:
checkpointer = ModelCheckpoint(filepath='/tmp/newweights.hdf5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=3)

In [18]:
model.fit_generator(generator=train_gen, steps_per_epoch=100, 
                    validation_data=val_gen, validation_steps=200, 
                    epochs=20, use_multiprocessing=True, callbacks=[checkpointer, earlystopper])

Epoch 1/20
100/100 [==============================] - 123s 1s/step - loss: 0.5750 - acc: 0.4318 - iou: 0.4128 - iou_flat: 0.3874 - dice: 0.4250 - dice_flat: 0.5523 - val_loss: 0.5463 - val_acc: 0.4600 - val_iou: 0.4442 - val_iou_flat: 0.4206 - val_dice: 0.4537 - val_dice_flat: 0.5898

Epoch 00001: val_loss improved from inf to 0.54629, saving model to /tmp/newweights.hdf5
Epoch 2/20
100/100 [==============================] - 118s 1s/step - loss: 0.5197 - acc: 0.4814 - iou: 0.4775 - iou_flat: 0.4561 - dice: 0.4803 - dice_flat: 0.6242 - val_loss: 0.5579 - val_acc: 0.4428 - val_iou: 0.4364 - val_iou_flat: 0.4063 - val_dice: 0.4421 - val_dice_flat: 0.5753

Epoch 00002: val_loss did not improve from 0.54629
Epoch 3/20
100/100 [==============================] - 114s 1s/step - loss: 0.5067 - acc: 0.4937 - iou: 0.4922 - iou_flat: 0.4743 - dice: 0.4933 - dice_flat: 0.6412 - val_loss: 0.5144 - val_acc: 0.4864 - val_iou: 0.4835 - val_iou_flat: 0.4632 - val_dice: 0.4856 - val_dice_flat: 0.6312

Ep

### VGG Segnet

In [ ]:
model = VGGSegnet(N_CLASSES, INPUT_HEIGHT, INPUT_WIDTH)

metrics = ['accuracy', iou, iou_flat, dice, dice_flat]


model.compile(optimizer='adam', 
              loss=dice_loss,  
              metrics=metrics)

model.fit_generator(generator=train_gen, steps_per_epoch=100, 
                    validation_data=val_gen, validation_steps=200, 
                    epochs=20, use_multiprocessing=True, callbacks=[checkpointer, earlystopper])

Epoch 1/20
100/100 [==============================] - 120s 1s/step - loss: 0.6060 - acc: 0.3981 - iou: 0.3896 - iou_flat: 0.3527 - dice: 0.3940 - dice_flat: 0.5125 - val_loss: 0.5353 - val_acc: 0.4650 - val_iou: 0.4630 - val_iou_flat: 0.4367 - val_dice: 0.4647 - val_dice_flat: 0.6048

Epoch 00001: val_loss did not improve from 0.50372
Epoch 2/20
100/100 [==============================] - 115s 1s/step - loss: 0.5141 - acc: 0.4863 - iou: 0.4843 - iou_flat: 0.4638 - dice: 0.4859 - dice_flat: 0.6315 - val_loss: 0.7701 - val_acc: 0.2260 - val_iou: 0.2210 - val_iou_flat: 0.1776 - val_dice: 0.2299 - val_dice_flat: 0.2966

Epoch 00002: val_loss did not improve from 0.50372
Epoch 3/20
100/100 [==============================] - 115s 1s/step - loss: 0.5079 - acc: 0.4924 - iou: 0.4912 - iou_flat: 0.4726 - dice: 0.4921 - dice_flat: 0.6397 - val_loss: 0.5577 - val_acc: 0.4435 - val_iou: 0.4367 - val_iou_flat: 0.4065 - val_dice: 0.4423 - val_dice_flat: 0.5745

Epoch 00003: val_loss did not improve fr

### FCN32